# Introduction to Asynchronous Programming
Asynchronous programming allows you to write code that can perform multiple operations at the same time. In Python, this is often facilitated through the `asyncio` library, which is built into the Python standard library. This approach is particularly useful in scenarios where tasks are IO-bound or when performing high-latency operations.

## Basic Asyncio Concepts
`asyncio` is a library to write concurrent code using the async/await syntax. An event loop is the central execution device provided by `asyncio`. This loop is capable of running asynchronous tasks and callbacks, performing network IO operations, and running subprocesses.

## Setting Up an Async Environment
To utilize `asyncio`, you need to establish an environment that supports asynchronous execution. Here's how to set up a basic async environment in Python:

In [6]:
# Import the asyncio library, which is used to write concurrent code using the async/await syntax in Python.
import asyncio

# Import the nest_asyncio module. This module patches asyncio to allow nested use of asyncio.run and related functions.
import nest_asyncio

# Apply the patch with nest_asyncio.apply(). This is necessary in environments like Jupyter notebooks where an event loop
# may already be running in the background. Without this patch, you would typically encounter a 'RuntimeError: This event 
# loop is already running' when trying to use asyncio.run().
nest_asyncio.apply()

# Define an asynchronous function named 'main'. Asynchronous functions are defined using 'async def' and can contain
# 'await' expressions.
async def main():
    # Print a greeting message to the console. This is a simple operation within the async function.
    print("Hello, asyncio!")

# Now that the environment is set up to handle it, use asyncio.run() to execute the main() function.
# asyncio.run() is used to run the top-level entry point “main” function and only takes a coroutine. It handles all the 
# event loop management such as opening and closing the loop, thus simplifying asyncio usage for basic scripts.
# It should not be used when an event loop is already running or for creating multiple event loops in an application.
asyncio.run(main())


Hello, asyncio!


## Simple Async Examples
Let's create a simple example to illustrate how asynchronous functions are defined and called in Python using `asyncio`.

In [5]:
# Define an asynchronous function named 'fetch_data' that accepts a URL as a parameter.
async def fetch_data(url):
    # Print a message indicating the start of data fetching from the specified URL.
    print(f"Starting to fetch data from {url}")
    
    # 'await asyncio.sleep(1)' pauses this coroutine, allowing the event loop to run other tasks
    # (such as starting another instance of 'fetch_data' for the next URL if not already started).
    # This simulates a delay of 1 second, like waiting for a network response, but does not block other tasks.
    await asyncio.sleep(1)  # Simulate a delay for fetching data
    
    # This line is executed after the sleep has finished. By this time, other coroutines might have started or even finished.
    print(f"Data fetched from {url}")

# A list of URLs from which data will be fetched. These could be API endpoints or any other URLs.
urls = ["http://example.com/api/data1", "http://example.com/api/data2"]

# Define the 'main' asynchronous function to manage and run tasks.
async def main():
    # Create a list of tasks. For each URL in the 'urls' list, 'asyncio.create_task()' is called,
    # which immediately schedules 'fetch_data(url)' to be run asynchronously.
    tasks = [asyncio.create_task(fetch_data(url)) for url in urls]
    
    # Here, 'asyncio.gather()' is used to wait for all the tasks to complete.
    # It allows the event loop to manage all tasks, switching between them whenever one of them is paused (e.g., during 'await').
    # Since 'asyncio.sleep(1)' in 'fetch_data' yields control back to the loop, it can start or resume other coroutines in the meantime.
    await asyncio.gather(*tasks)

# Run the 'main' function, which is the entry point for the asyncio program.
# 'asyncio.run()' manages the event loop, which handles all the asynchronous tasks.
asyncio.run(main())


Fetching data...
Data fetched!


In this code:

* When fetch_data(url) for "http://example.com/api/data1" starts and hits await asyncio.sleep(1), it pauses, and the control is yielded back to the asyncio event loop.
* The event loop then checks if there are other tasks ready to run or resume. Since the tasks have been created for each URL and are managed by the event loop, it starts the next available task, which would be fetch_data(url) for "http://example.com/api/data2".
* If both tasks are in the sleep state, the event loop effectively handles other tasks (if any) or simply waits until the sleep durations expire.
* As each task completes its sleep, it resumes execution to complete its remaining work, and all tasks must complete before asyncio.gather() finishes.


This model is very efficient for handling I/O-bound operations, such as network requests, because it maximizes the usage of the program's running time, keeping the operation as asynchronous and non-blocking as possible.

## Data Fetching
Fetching data asynchronously can greatly improve the performance of your application when dealing with API requests or reading from files. Here's how you can fetch data asynchronously from a mock API.

In [8]:
# Import the asyncio library which provides support for asynchronous I/O, event loops, coroutines, tasks, and more.
import asyncio

# Define an asynchronous function 'fetch_data' that takes a URL as an argument.
async def fetch_data(url):
    # Print a message indicating that data fetching has started for the given URL.
    print(f"Starting to fetch data from {url}")
    
    # 'await' suspends execution of the current function (fetch_data),
    # allowing the program to execute other tasks while waiting for some I/O operation,
    # here simulated as a 1-second sleep.
    await asyncio.sleep(1)  # Simulate a delay for fetching data
    
    # Once the await operation (sleep) is completed after 1 second, execution resumes and prints that the data has been fetched.
    print(f"Data fetched from {url}")

# List of URLs to fetch data from. Each URL represents a distinct data endpoint.
urls = ["http://example.com/api/data1", "http://example.com/api/data2"]

# Define the main asynchronous function that manages higher-level tasks.
async def main():
    # Create a list of task objects by iterating over each URL and creating a new task for each call to fetch_data.
    # asyncio.create_task() schedules the execution of a coroutine and immediately returns a Task object.
    tasks = [asyncio.create_task(fetch_data(url)) for url in urls]
    
    # asyncio.gather() is used to schedule the execution of these tasks concurrently.
    # It awaits on all provided tasks to complete. The '*' operator unpacks the list into separate arguments.
    # This line effectively pauses 'main()' until all 'fetch_data' tasks have completed.
    await asyncio.gather(*tasks)

# Run the main function using asyncio.run, which creates an event loop and executes the main function within it.
# This call also handles cleaning up the event loop after the function completes.
asyncio.run(main())


Starting to fetch data from http://example.com/api/data1
Starting to fetch data from http://example.com/api/data2
Data fetched from http://example.com/api/data1
Data fetched from http://example.com/api/data2


## Asynchronous Data Processing
Using `asyncio` in data processing can help manage both IO-bound and CPU-bound tasks more efficiently. This example shows how you can process data asynchronously.

In [9]:
# Import necessary libraries. Pandas for data manipulation and analysis, and NumPy for numerical operations.
import pandas as pd
import numpy as np

# Define an asynchronous function 'load_data' which simulates the loading of data asynchronously.
async def load_data():
    # Await a sleep call to simulate an I/O-bound operation, such as reading data from a file or a database.
    # This could represent waiting for data to be read from disk or received over the network.
    await asyncio.sleep(2)
    
    # Return a pandas DataFrame created with random integers between 0 and 100.
    # This DataFrame simulates actual data and consists of 100 rows and 4 columns.
    return pd.DataFrame(np.random.randint(0, 100, size=(100, 4)), columns=list('ABCD'))

# Define another asynchronous function 'process_data' that takes a DataFrame as input and processes it.
async def process_data(data):
    # Print a message indicating that data processing has started.
    print("Processing data...")
    
    # Await a sleep call to simulate a CPU-bound task, like a complex calculation.
    # This could represent time-consuming computations or transformations on the data.
    await asyncio.sleep(2)
    
    # Return the statistical description of the DataFrame, which includes count, mean, std, min, quartiles, and max.
    return data.describe()

# Define the main coroutine, which orchestrates the loading and processing of data.
async def main():
    # Await the 'load_data' function to finish and store its result in the variable 'data'.
    # This represents receiving the dataset that needs processing.
    data = await load_data()
    
    # Await the 'process_data' function with the loaded data and store its result in the variable 'result'.
    # This represents the processed data being ready for further use or analysis.
    result = await process_data(data)
    
    # Print the processed data's descriptive statistics to the console.
    print(result)

# Start the event loop and run the main coroutine.
# 'asyncio.run' manages the creation and destruction of the event loop, ensuring everything executes as expected.
asyncio.run(main())


Processing data...
                A          B           C           D
count  100.000000  100.00000  100.000000  100.000000
mean    52.830000   43.99000   51.240000   53.370000
std     27.945493   29.07861   28.470832   28.562055
min      0.000000    1.00000    1.000000    0.000000
25%     32.750000   18.75000   25.000000   33.500000
50%     56.000000   42.00000   53.500000   52.500000
75%     73.000000   70.00000   76.250000   77.250000
max     98.000000   99.00000   98.000000   99.000000


## Concurrency in Data Analysis
Integrating `asyncio` with libraries like pandas can enhance performance significantly by handling tasks concurrently. Here's an example of using asyncio with pandas for a typical data analysis workflow.

In [ ]:
# This is a simple async function
import asyncio

async def hello_async():
    await asyncio.sleep(1)
    print("Hello, Async World!")

# Running the function using asyncio
asyncio.run(hello_async())


# Enhanced Async Features in Python

This section introduces a simplified overview of asynchronous features in Python, derived from an extensive tutorial and practical examples that make the concept more accessible.

## Understanding Asynchronous vs. Synchronous Programming

Asynchronous programming allows a program to handle multiple operations at once rather than waiting for a task to complete before moving on to the next one. This is useful for IO-bound tasks where the program can perform other operations while waiting for IO operations to complete, such as web requests or file reads/writes.

## Practical Examples of Asyncio in Python

Here are some practical examples demonstrating basic asyncio usage in Python, focusing on tasks and the event loop.

In [10]:
# Import the asyncio module, which is used to write concurrent code using the async/await syntax.
import asyncio

# Define an asynchronous function named 'basic_async_example'. Asynchronous functions are defined using 'async def',
# and they enable the function to use 'await' for asynchronous operations.
async def basic_async_example():
    # Print a statement indicating the start of an operation, useful for tracking the progress of the code.
    print('Start of basic example')
    
    # Use 'await' to pause the execution of the current coroutine, allowing the event loop to run other tasks.
    # 'asyncio.sleep(1)' simulates a pause or delay for 1 second, mimicking an I/O operation like reading from a file or a network request.
    # This is a non-blocking call, meaning it does not block the execution of other asynchronous tasks in the event loop.
    await asyncio.sleep(1)
    
    # After the 1-second delay, the execution resumes and the following print statement is executed.
    # This indicates the end of the example.
    print('End of basic example')

# 'asyncio.run()' is used to run the top-level coroutine, here 'basic_async_example'.
# It creates an event loop, runs the passed coroutine to completion, and closes the loop.
# This is the preferred way to run a complete asynchronous program.
asyncio.run(basic_async_example())


Start of basic example
End of basic example


## Async Event Loop and Task Creation

The event loop is the core of asyncio's architecture, handling the execution of multiple tasks. Here’s how you can create and manage async tasks in Python:

In [7]:
# Import the asyncio module, which provides support for writing concurrent code using coroutines, multiplexing I/O access, etc.
import asyncio

# Define the first asynchronous function 'task1'.
async def task1():
    # Print a message indicating the start of task 1.
    print("Task 1 started..")
    # The await keyword is used to pause the execution of task1 while waiting for the asyncio.sleep(2) to complete.
    # This simulates a delay of 2 seconds, perhaps mimicking some I/O operation or long-running computation.
    await asyncio.sleep(2)
    # After the sleep completes, print a message indicating the end of task 1.
    print("Task 1 Ended")

# Define the second asynchronous function 'task2'.
async def task2():
    # Similar to task1, print a message at the start.
    print("Task 2 started..")
    # Pause the execution of task2 for 3 seconds. This represents a longer operation compared to task1.
    await asyncio.sleep(3)
    # Once the sleep is over, indicate the end of task 2.
    print("Task 2 Ended")

# Define the third asynchronous function 'task3'.
async def task3():
    # Announce the start of task 3.
    print("Task 3 started..")
    # Pause the execution of task3 for 1 second, the shortest wait time among the three tasks.
    await asyncio.sleep(1)
    # Mark the completion of task 3.
    print("Task 3 Ended")

# Define a function 'run_tasks' to manage and run the defined tasks concurrently.
async def run_tasks():
    # asyncio.gather() is used to run multiple tasks concurrently. It takes multiple awaitable objects (coroutines, Futures, etc.)
    # and schedules them to run concurrently. When using gather, if one of the tasks raises an exception, all are cancelled.
    await asyncio.gather(
        task1(),
        task2(),
        task3(),
    )
    # Once all tasks are complete, print a confirmation message.
    print("All tasks completed.")

# The asyncio.run() function is used to execute the run_tasks coroutine, which in turn runs all other tasks.
# It manages the creation, running, and closing of the event loop, simplifying the execution of the async functions.
asyncio.run(run_tasks())


Task 1 started..
Task 2 started..
Task 3 started..
Task 3 Ended
Task 1 Ended
Task 2 Ended
All tasks completed.


Expected Outcome and Execution Flow:
1) Start Execution: The run_tasks() coroutine is called by asyncio.run(), which sets up and manages the event loop.
2) Concurrent Execution:
    * Task 1 starts and prints "Task 1 started..", then it sleeps for 2 seconds.
    * Almost simultaneously, Task 2 and Task 3 start as well. They print their respective start messages and enter their sleep periods.
    * Task 3 finishes first since it only sleeps for 1 second. It prints "Task 3 Ended".
    * After Task 3, Task 1 completes its 2-second sleep and prints "Task 1 Ended".
    * Finally, Task 2, which has the longest sleep time, finishes and prints "Task 2 Ended".
3) Completion:
    * Once all tasks are finished, asyncio.gather() completes, and "All tasks completed." is printed.
4) Program Ends: With the completion of run_tasks(), asyncio.run() finalizes by closing the event loop.